In [ ]:
# to check, whether the results depend on shuffling
# test on the smallest set
%matplotlib inline
import matplotlib.pyplot as plt
import subprocess
import os
import random
import shutil
from collections import Counter

DATASET_DIR = "../tests/input_files/test_100_50_25/"
TEMP_DIR = "temp"
DATASET_FILES = os.listdir(DATASET_DIR)
ANSWER = 25
STRNUM = 50
STRLEN = 100
os.mkdir(TEMP_DIR) if not os.path.isdir(TEMP_DIR) else None

In [ ]:
# create shuffled files
to_check = random.choice(DATASET_FILES)
to_check_path = os.path.join(DATASET_DIR, to_check)
replicate_min = {}  # number of replicate -> minimal answer

rep_files = []
shuffle_cmd_templ = "../shuffle_input.py " + to_check_path + " {} -o {} -c"
for i in range(1, STRNUM):
    out_file = os.path.join(TEMP_DIR, "rep_{}.txt".format(i))
    shuffle_cmd = shuffle_cmd_templ.format(i, out_file)
    subprocess.call(shuffle_cmd, shell=True)
    rep_files.append(out_file)

In [ ]:
# call CSP for each file
# a long stage
csp_cmd_templ = "../CSP {} {}"

for num, rep in enumerate(rep_files):
    # get the minimal answer for each
    for i in range(ANSWER, STRLEN):
        csp_cmd = csp_cmd_templ.format(rep, i)
        csp_out = subprocess.check_output(csp_cmd, shell=True).decode("utf-8").split("\n")[1]
        if csp_out == "False":
            continue
        # write the result then
        replicate_min[num] = i
        break

In [ ]:
# visualize it
min_times = Counter()
for v in replicate_min.values():
    min_times[v] += 1
times_sort = sorted(min_times.keys())
for t in range(times_sort[0], times_sort[-1] + 1):
    app_ = min_times.get(t) if min_times.get(t) else 0
    print("{}:\n{}".format(t, "*" * app_))
# defenitely, algorithm is init string - dependent

In [ ]:
# maybe get some statistics?
rep_zeros = {}
rep_k = {}
nums = range(len(rep_files))
maxes = []
mins = []
ks = []
deltas = []
aves = []

for num, rep in enumerate(rep_files):
    # get the minimal answer for each
    f = open(rep, "r")
    lines = [l[:-1] for l in f.readlines()]
    f.close()
    
    k = replicate_min[num]
    print("\nReplicate num {} K: {}".format(num, k))
    zeros = sorted([line.count("0") for line in lines])
    rep_zeros[num] = zeros[1:]
    rep_k[num] = k
    ks.append(k)
    max_zeros = zeros[-1]
    min_zeros = zeros[1]  # because in 0 -> 0
    maxes.append(max_zeros)
    mins.append(min_zeros)
    deltas.append(delta)
    delta = max_zeros - min_zeros
    ave = sum(zeros) / len(zeros)
    aves.append(ave)
    print("Min zeros: {}; Max: {}, D: {}".format(min_zeros, max_zeros, delta))

In [ ]:
# visualize zero distrs
# maybe it is the key?
fig = plt.figure(figsize=(20, 100))
rep_num = len(rep_files)
columns = 2
rows = (rep_num // 2) + (rep_num % 2)
zeros_num = len(rep_zeros[0])
bins = zeros_num // 4
keys_ord = sorted([k for k in rep_k.keys()], key=lambda x: rep_k[x])

for i in range(1, rep_num + 1):
    r_i = keys_ord[i - 1]
    title = "Replicate {}/ K = {}".format(r_i, rep_k[r_i])
    ax1 = fig.add_subplot(rows, columns, i)
    plt.hist(rep_zeros[r_i], bins=bins)
    ax1.title.set_text(title)


In [ ]:
# another visualisation?
fig = plt.figure(figsize=(15, 15))
ax1 = fig.add_subplot(2, 2, 1)
ax2 = fig.add_subplot(2, 2, 2)
ax3 = fig.add_subplot(2, 2, 3)
ax4 = fig.add_subplot(2, 2, 4)

ax1.scatter(ks, maxes, s=2, color="red")
ax1.set_xlabel("K")
ax1.set_ylabel("Max zeros")

ax2.scatter(ks, mins, s=2, color="green")
ax2.set_xlabel("K")
ax2.set_ylabel("Min zeros")

ax3.scatter(ks, deltas, s=2, color="blue")
ax3.set_xlabel("K")
ax3.set_ylabel("Min max delta")

ax4.scatter(ks, aves, s=2, color="#222222")
ax4.set_xlabel("K")
ax4.set_ylabel("Average")

plt.show()

In [ ]:
# let's have a look at this...
from collections import defaultdict
fig = plt.figure(figsize=(10, 10))
K_range = list(range(min(ks), max(ks) + 1))
K_range_size = len(K_range)
max_R = 255
min_R = 50
G = "10"
B = "10"
C_ = "#{}{}{}"
k_to_m_m = defaultdict(list)
for elem in zip(ks, maxes, mins):
    k_to_m_m[elem[0]].append((elem[1], elem[2]))

k_keys = sorted(set(ks))
step = int((max_R - min_R) / K_range_size)
for num, k_ in enumerate(k_keys):
    R = hex(min_R + step * num)[2:].upper()
    k_vals = k_to_m_m[k_]
    _max = [x[0] for x in k_vals]
    _min = [x[1] for x in k_vals]
    color = C_.format(R, G, B)
    plt.scatter(_max, _min, s=3, color=color)
plt.xlabel("Max zeros")
plt.ylabel("Min zeros")
plt.show()

In [ ]:
shutil.rmtree(TEMP_DIR) if os.path.isdir(TEMP_DIR) else None